# PINN Evaluation

In [1]:
import numpy as np
import pandas as pd
import torch
from pinn import PINN
import os

In [2]:
data_dir = "/Users/ggito/repos/pinns/data/front_wing"
model_dir = os.path.join(data_dir, "models")
points_file_name = "points_final.csv"
model_name = "v8"
model_checkpoint_num = 60

In [3]:
wing_df = pd.read_csv(os.path.join(data_dir, points_file_name))

In [4]:
device = torch.device("cpu")

In [5]:
x_min, y_min, z_min = wing_df.min()
x_max, y_max, z_max = wing_df.max()

overall_min = min(x_min, y_min, z_min)
overall_max = max(x_max, y_max, z_max)

xyz_range = (overall_min, overall_max)

In [6]:
# load model
input_dim = 4
output_dim = 4
# hidden_units = [1024, 1024, 1024]
hidden_units = [2058, 2058, 2058]
# hidden_units = [1024, 1024, 1024, 1024, 1024]
pinn = PINN(input_dim, output_dim, hidden_units, model_name=None).to(device)
pinn, _ = pinn.load_checkpoint_num(optimizer=None, checkpoint_dir=model_dir, model_name=model_name, checkpoint_num=model_checkpoint_num, mode='test')
pinn.eval()

=> loading checkpoint '/Users/ggito/repos/pinns/data/front_wing/models/v8/60.pt'
=> loaded checkpoint '/Users/ggito/repos/pinns/data/front_wing/models/v8/60.pt' (epoch 60)


PINN(
  (layers): ModuleList(
    (0): Linear(in_features=4, out_features=2058, bias=True)
    (1-2): 2 x Linear(in_features=2058, out_features=2058, bias=True)
    (3): Linear(in_features=2058, out_features=4, bias=True)
  )
)

In [7]:
# testing domain
x_max = 1
y_max = 1
z_max = 1
t_max = 2 # TODO: set t>2 to test outside of training domain

Nx = 200
Ny = 200
Nz = 200
Nt = 200

dx = x_max / (Nx - 1)
dy = y_max / (Ny - 1)
dz = z_max / (Nz - 1)
dt = t_max / (Nt - 1)

x_test = np.linspace(0, x_max, Nx)
y_test = np.linspace(0, y_max, Ny)
z_test = np.linspace(0, z_max, Nz)
t_test = np.linspace(0, t_max, Nt)

x_grid, y_grid, z_grid = np.meshgrid(x_test, y_test, z_test, indexing='ij')

In [8]:
pinn.eval_pinn(s_df=wing_df, Nf=10000, N0=10000, Nb=10000, Ns=10000,
               x_max=x_max, y_max=y_max, z_max=z_max, t_max=t_max,
               in_velocity=1, mu=1, rho=1,
               device=device)

(tensor(17.5380, grad_fn=<AddBackward0>),
 tensor(2.8642, grad_fn=<AddBackward0>),
 tensor(1.8327, grad_fn=<MulBackward0>),
 tensor(1.0601, grad_fn=<MulBackward0>),
 tensor(3.4177, grad_fn=<MulBackward0>),
 tensor(0.2177, grad_fn=<MulBackward0>),
 tensor(2.1076, grad_fn=<MulBackward0>),
 tensor(2.1155, grad_fn=<MulBackward0>),
 tensor(1.2026, grad_fn=<MulBackward0>),
 tensor(0.9891, grad_fn=<MulBackward0>),
 tensor(1.7308, grad_fn=<MulBackward0>))